OpenCV notebook - ©Arkaprabha Majumdar

In [1]:
import cv2
import numpy as np

In [ ]:
image=cv2.imread( #reads images
    '/home/arkaprabham/Desktop/Deep_Learning_Udemy/img/quotes.jpg')
#The image is HUGE ! so we need to scale it first.
image_scaled=cv2.pyrDown(image)

## #1. Erosion and Dilation
In openCV, for black on white text:
    dilation thins the text
    erosion bolds the text

In [ ]:
# Creating kernel 
kernel = np.ones((5, 5), np.uint8) 
# Using cv2.erode() method  
eroded = cv2.erode(image_scaled, kernel)
cv2.imshow("Eroded image",eroded)
dilated = cv2.dilate(eroded, kernel)
cv2.imshow("Dilated image",dilated)
cv2.waitKey(0)
cv2.destroyAllWindows()

Here, above... I had to pass the eroded image to the dilate function
because the text disappears on dilating original image

#### Sidenote: Do check if the image has properly loaded if you get an assertion error
In fact, what we did above is called CLOSING.

In [ ]:
opening=cv2.morphologyEx(image_scaled,cv2.MORPH_OPEN,kernel)
closing=cv2.morphologyEx(image_scaled,cv2.MORPH_CLOSE,kernel)
cv2.imshow("Opened image",opening)
cv2.imshow("Closed image",closing)
cv2.waitKey(0)
cv2.destroyAllWindows()

As you can see, the opened image removes the text in THIS CASE.

#### If original text is already fat, OPEN !
#### If original text is already thin, CLOSE !

## #2. Edge Detection

3 types of edge detection algorithms:
    Sobel
    Laplacian
    Canny

In [ ]:
animal_gray=cv2.imread('/home/arkaprabham/Desktop/Deep_Learning_Udemy/img/big cat.jpg',0)
#Let's scale it down
animal_gray=cv2.pyrDown(animal_gray)
sobel_x = cv2.Sobel(animal_gray,cv2.CV_64F,0,1,ksize=5) #horizontal edges
sobel_y = cv2.Sobel(animal_gray,cv2.CV_64F,1,0,ksize=5) #vertical edges

#We can combine both
sobel_both=cv2.bitwise_or(sobel_x,sobel_y)

Laplacian=cv2.Laplacian(animal_gray,cv2.CV_64F) #no params

canny=cv2.Canny(animal_gray,20,170)
#values for the thresholds can be tweaked for better results


cv2.imshow("SOBEL X",sobel_x)
cv2.imshow("SOBEL Y",sobel_y)
cv2.imshow("SOBEL Combined",sobel_both)
cv2.imshow("LAPLACIAN",Laplacian)
cv2.imshow("CANNY",canny)

cv2.waitKey(0)
cv2.destroyAllWindows()

As you can see, all the outputs are different yet we can make it out that's a big lemur pup.

## #4. Perspective Transforms

In [ ]:
#Let's import another picture for this
book_image=cv2.imread('/home/arkaprabham/Desktop/Deep_Learning_Udemy/img/book of romans.jpg',0)
book_image=cv2.resize(book_image,None,fx=.9,fy=.9)
#The image was exceeding the size of my screen, so I scaled by .9
cv2.imshow("Book of Romans",book_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#Next we'll note down the book corner points
'''338,167
315,335
517,179
543,338'''
points_page=np.float32([[337,176],
                       [315,337],
                       [513,181],
                       [543,339]])

#then we input the points for a regular A4 page
points_A4=np.float32([[0,0],
                      [0,594],
                      [420,0],
                      [420,594]])

#Next create the perspective transform matrix
M=cv2.getPerspectiveTransform(points_page,points_A4)

#Now warp!
warped_page=cv2.warpPerspective(book_image,M,(420,594))

cv2.imshow("Book of Romans",book_image)
cv2.imshow("Corrected book of Romans",warped_page)
cv2.waitKey(0)
cv2.destroyAllWindows()

Let's make a image manipulator for our webcam.
# Project : Webcam

In [ ]:
def sketch(image):
    #change the input image to grascale
    image_grayscale=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    
    #gaussian blur - to denoise the image.
    '''You can check the effect of not denoising by making the next line uncommented'''
    denoised_img=cv2.GaussianBlur(image_grayscale,(5,5),0)
    #denoised_img=image_grayscale
    
    
    #we need to detect edges for whatever we do, so
    edges=cv2.Canny(denoised_img,20,50)
    
    #apply thresholding to get various sketch effects
    ret,mask=cv2.threshold(edges,70,255,cv2.THRESH_BINARY_INV)
    return mask

#Next we take webcam feed as frames using VideoCapture object
cap=cv2.VideoCapture(0)

#Then we infinite loop each single frame to create video
#Without loop, its only one image
while True:
    _,frame=cap.read()
    cv2.imshow("Video Capture threshold feed",sketch(frame))
    if cv2.waitKey(1)==13: #13 is the Enter key. you can put any key here
        break
        
cap.release()
cv2.destroyAllWindows()

In [ ]:
image=cv2.pyrDown(cv2.imread('/home/arkaprabham/Desktop/Deep_Learning_Udemy/img/Crowd.jpg'))
#This is a little tough photo because it has so many people.

#So first we convert the image to grayscale
image_grayscale=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

#Then, to reduce noise we use Gaussian Blur
denoised_img=cv2.GaussianBlur(image_grayscale,(5,5),0)

#Detecting edges using Canny.This is an important step. Try tweaking the parameter values 
edges=cv2.Canny(denoised_img,160,200)

#the findContours function in OpenCV returns a list of list of boundary points, and their hierarchy
contours,hierarchy=cv2.findContours(edges,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
'''RETR is a retrieval parameter signifying hierarchy:
            RETR_EXTERNAL signifies that we want the outer contours only.
            RETR_LIST retrieves all contours.
            RETR_COMP retrieves 2-level hierarchies.
            RETR_TREE retrieves all hierarchies.
'''

''' CHAIN_APPROX_NONE returns all boundary points
    CHAIN_APPROX_SIMPLE returns only corner points
'''

cv2.imshow("contour",cv2.drawContours(image,contours,-1,(0,255,0),3))
# "-1" gives all contours, '1' gives first contour, '2' gives second, etc...
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def contour(image):
    #change the input image to grascale
    image_grayscale=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    
    #gaussian blur - to denoise the image.
    '''You can check the effect of not denoising by making the next line uncommented'''
    denoised_img=cv2.GaussianBlur(image_grayscale,(5,5),0)
     

    #we need to detect edges for whatever we do, so
    edges=cv2.Canny(denoised_img,10,50)
    
    #contour
    contours,hierarchy=cv2.findContours(edges,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    return contours

#Next we take webcam feed as frames using VideoCapture object
cap=cv2.VideoCapture(0)

#Then we infinite loop each single frame to create video
#Without loop, its only one image
while True:
    _,frame=cap.read()
    contours=contour(frame)
    cv2.imshow("contour",cv2.drawContours(frame,contours,-1,(0,255,0),3))
    if cv2.waitKey(1)==13: #13 is the Enter key. you can put any key here
        break
        
cap.release()
cv2.destroyAllWindows()

And like that...we can now apply every effect we learn to our webcam !

In [ ]:
image.shape